In [7]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import requests
import numpy as np

In [11]:
%%time

url_csv_bic= '../datasets/bicimad_stations.csv' # esto se tendría que desconectar si pudiéramos conectarnos en clase
dfb = pd.read_csv(url_csv_bic, sep=',') # esto se tendría que desconectar si pudiéramos conectarnos en clase
df_bic = dfb[['name', 'address', 'geometry.coordinates']]
df_bic[['location.longitude.bic', 'location.latitude.bic']]= df_bic['geometry.coordinates'].str.split(',', expand=True)
df_bic['location.longitude.bic'] = df_bic['location.longitude.bic'].apply(lambda x: float(x[1:]))
df_bic['location.latitude.bic'] = df_bic['location.latitude.bic'].apply(lambda x: float(x[:-1]))
df_bic = df_bic[['name', 'address', 'location.latitude.bic', 'location.longitude.bic']]
endpoint = 'https://datos.madrid.es/egob'
url = '/catalogo/300356-0-monumentos-ciudad-madrid.json'
response = requests.get(endpoint + url)
json_data = response.json()
dfm = pd.DataFrame(pd.json_normalize(json_data['@graph']))
df_monu = dfm[['title', 'address.street-address', 'location.latitude' , 'location.longitude']]
df_monu = df_monu.dropna()
df_bic['key'] = 0
df_monu['key'] = 0
df_pre = df_bic.merge(df_monu, on='key', how='left')
url_df_pre = '../datasets/df.csv'
df_def = pd.read_csv(url_df_pre, sep=',')
zz5 = df_def.sort_values(by = ['title', 'Distancia']).rename(columns={'title':'Place of Interest', 
                                                                      'address.street-address':'Place address',
                                                                     'name':'BiciMAD station',
                                                                      'address':'Station Location'})
zz5['Type of place'] = np.nan
zz5['Type of place'] = zz5['Type of place'].fillna('Monumento de la ciudad de Madrid')
df_final = zz5[['Place of Interest', 'Type of place', 'Place address', 'BiciMAD station', 'Station Location']]
df_final['BiciMAD station'] = df_final['BiciMAD station'].apply(lambda x: x.split('-')[1])
is_poi = df_final.loc[:, 'Place of Interest'] == 'A los Abuelos'
df_poi = df_final.loc[is_poi]
df_poi = df_poi.reset_index(drop=True)
print(df_poi.head(1))
csv_poi = df_poi.head(1).to_csv('../datasets/specific_point_of_interest.csv', index=False)
poi_uniques = df_final['Place of Interest'].unique().tolist()
test = df_final.loc[:, 'Place of Interest'] == 'A Neruda-El Ser alado'
test1 = df_final.loc[test]
test1 = test1.reset_index(drop=True)
test1 = test1.head(0)
for i in poi_uniques:
    test_2 = df_final.loc[:, 'Place of Interest'] == i
    test2 = df_final.loc[test_2]
    test2 = test2.reset_index(drop=True)
    test2 = test2.head(3)
    test1 = test1.append(test2, ignore_index=True)
print(test1.head())
csv_full = test1.to_csv('../datasets/all_points_of_interests.csv', index=False)

<timed exec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

  Place of Interest                     Type of place Place address  \
0     A los Abuelos  Monumento de la ciudad de Madrid     C Alicún    

    BiciMAD station               Station Location  
0   Manuel Caldeiro  Paseo de la Castellana nº 298  
               Place of Interest                     Type of place  \
0          A Neruda-El Ser alado  Monumento de la ciudad de Madrid   
1          A Neruda-El Ser alado  Monumento de la ciudad de Madrid   
2          A Neruda-El Ser alado  Monumento de la ciudad de Madrid   
3  A las víctimas del Holocausto  Monumento de la ciudad de Madrid   
4  A las víctimas del Holocausto  Monumento de la ciudad de Madrid   

              Place address      BiciMAD station  \
0  Gta José Luis Ozores 116            Concordia   
1  Gta José Luis Ozores 116      Sierra Toledana   
2  Gta José Luis Ozores 116        Doctor Lozano   
3     JARDIN TRES CULTURAS       Carlota O’Neill   
4     JARDIN TRES CULTURAS    Gutierre de Cetina   

                 

In [26]:
def connec_bic(bbdd, path='../datasets/bicimad_stations.csv'): #las dos opciones serían 'mysql' o 'csv' y se podría cambiar el path
    if bbdd == 'mysql':
        ip_user = input('Con qué IP te quieres conectar (clase, casa o nueva)?: ')
        if ip_user == 'clase':
            ip = '173.201.189.217'
            connection_string = 'mysql+pymysql://ironhack_user:%Vq=c>G5@'+ip+'/BiciMAD'
            engine = create_engine(connection_string)
            inspector = inspect(engine)
            dfb = pd.read_sql_query("SELECT * FROM bicimad_stations", engine)
            return dfb            
        elif ip_user == 'casa':
            ip = '92.172.38.71'
            connection_string = 'mysql+pymysql://ironhack_user:%Vq=c>G5@'+ip+'/BiciMAD'
            engine = create_engine(connection_string)
            inspector = inspect(engine)
            dfb = pd.read_sql_query("SELECT * FROM bicimad_stations", engine)
            return dfb
        elif ip_user == 'nueva':
            ip = input('Facilita una dirección de ip válida (clase, casa o nueva): ')
            connection_string = 'mysql+pymysql://ironhack_user:%Vq=c>G5@'+ip+'/BiciMAD'
            engine = create_engine(connection_string)
            inspector = inspect(engine)
            dfb = pd.read_sql_query("SELECT * FROM bicimad_stations", engine)
            return dfb
        else:
            return 'Me da la sensación que no sabes dónde estás. ¿Estás en clase, en casa o introducir una nueva?
    elif bbdd == 'csv':
        dfb = pd.read_csv(path, sep=',')
        return dfb
    else:
        return 'Por el momento las dos únicas opciones son: mysql o csv'

In [29]:
ip_user = input('Te quieres conectar a la bbdd msql o al csv?: ')
connec_bic(ip_user)

Te quieres conectar a la bbdd msql o al csv?: csv


,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]"
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]"
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]"
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]"
